# Introduction to Deep Learning: Final Project

**Submitted by:**

Roei Matz       205871478

Yotam Silverman 313532418

# **Project Desctiption**

In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import collections

import torch
import torch.nn as nn
import torch.optim as optim